# Genetic algorithm-based heuristic for feature selection in credit risk assessment

## Questions :

### The Search Space Reduction 

* Means of the filter techniques for feature ranking: Information gain, Gain ratio,Gini index and Correlation => How we should create initial Chromosomes exactly ?
* How we should Initialize initial population ?
* What is Feature Ranking Exactly ? 
* What tik of essay means ?
* what it means by p_initial, i think it means other part of initial population

## Explanation
* First Hypothesis: We should first find reduce features to reduce features subset, then we should create initial_population based on this features.
* Second Hypothesis: We should Initial our first population just from reduced feature subset.
* Third Hypothesis: If We Initialized Chromosomes from reduced features subset and we couldn't reach minimum of features then we fill remainder by features that are not included in reduced feature subset

## Steps 
* Random R between 5 and 12
* Probabilty = rank/sum(ranks)
* Roulette wheel

## Ideas
* GA without filtering
* This method without current solution
* This method 2 times with current solution
* Rank Selection 

First we import libraries ,we will be needed through this notebook : 

In [19]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from numpy.random import rand as random
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import adam
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from keras.layers import Input

## Reduced Features Configurations

In [20]:
reduced_feature_config = {
    'IG': [ 0,  1,  2,  6,  9, 10, 11, 19, 20, 21, 22, 24],
    'gain_ratio': [ 0,  1,  2,  4,  9, 10, 19, 20, 21, 22, 24, 29],
    'correlation': [ 0,  1,  2,  4,  6,  9, 10, 11, 19, 20, 22, 24],
    'voting': [ 0,  1,  2,  9, 10, 19, 20, 22, 24],
    'current_solution': [ 0,  1,  2,  3, 10, 12, 13, 14, 17, 19, 20, 27, 29]
}

## Calculating Union Of reduced_feature_config And every feature rank

In [21]:
reduced_feature_subset = []
for key in reduced_feature_config : 
    reduced_feature_subset += reduced_feature_config[key]
reduced_feature_subset_rank = {}
for feature in reduced_feature_subset:
    if feature in reduced_feature_subset_rank:
        reduced_feature_subset_rank[feature] += 1
    else :
        reduced_feature_subset_rank[feature] = 1
reduced_feature_subset = sorted(list(set(reduced_feature_subset)))
reduced_feature_subset_rank

{0: 5,
 1: 5,
 2: 5,
 3: 1,
 4: 2,
 6: 2,
 9: 4,
 10: 5,
 11: 2,
 12: 1,
 13: 1,
 14: 1,
 17: 1,
 19: 5,
 20: 5,
 21: 2,
 22: 4,
 24: 4,
 27: 1,
 29: 2}

## Configes

In [22]:
genetic_config = { 
    'maxFeatureNum': 12,
    'minFeatureNum': 5,
    'popSize': 50,
    'trueProb': .5,
    'cProb': .9,
    'mProb': .01,
    'generationNum': 50,
    'tPressure': .1,
    'genNum': 0  
}
nn_config = {
    'lr': np.random.uniform(0.3, 1.0),
    'trainCycles': np.random.uniform(300, 600),
    'm': np.random.uniform(0.2, 0.7)
}
skf = StratifiedKFold(n_splits=10)
nn_config

{'lr': 0.6083023490593448,
 'm': 0.3091542232516784,
 'trainCycles': 533.2731552554148}

In [23]:
nn_config

{'lr': 0.6083023490593448,
 'm': 0.3091542232516784,
 'trainCycles': 533.2731552554148}

## Address Of Datasets 

In [7]:
data_root = 'data'
german_data =os.path.join(data_root,'GermanCreditInput.xls')
german_label = os.path.join(data_root,'GermanCreditOutputClass1columnknn.xls')
australian_dataset = os.path.join(data_root,'australian dataset.xlsx')
x_train = pd.read_excel(german_data,header=None)
y_train = pd.read_excel(german_label,header=None)
x_train.iloc[np.full(30,True)]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,6,4,0,0,0,1,0,0,1169,...,0,67,0,0,1,2,2,1,1,0
1,1,48,2,0,0,0,1,0,0,5951,...,0,22,0,0,1,1,2,1,0,0
2,3,12,4,0,0,0,0,1,0,2096,...,0,49,0,0,1,1,1,2,0,0
3,0,42,2,0,0,1,0,0,0,7882,...,0,45,0,0,0,1,2,2,0,0
4,0,24,3,1,0,0,0,0,0,4870,...,1,53,0,0,0,2,2,2,0,0
5,3,36,2,0,0,0,0,1,0,9055,...,1,35,0,0,0,1,1,2,1,0
6,3,24,2,0,0,1,0,0,0,2835,...,0,53,0,0,1,1,2,1,0,0
7,1,36,2,0,1,0,0,0,0,6948,...,0,35,0,1,0,1,3,1,1,0
8,3,12,2,0,0,0,1,0,0,3059,...,0,61,0,0,1,1,1,1,0,0
9,1,30,4,1,0,0,0,0,0,5234,...,0,28,0,0,1,2,3,1,0,0


## Chromosome Implementation 

In [9]:
class ToolBox :
    @staticmethod
    def save_item(path,item):
        with open(path + '.pkl', 'wb') as f:
            pickle.dump(item, f, pickle.HIGHEST_PROTOCOL)
    
    @staticmethod
    def load_item(path):
        with open(path + '.pkl', 'rb') as f:
            return pickle.load(f)

In [35]:
class Chromosome : 
    chromosomes = {} # Every Computed Chromosome will be save in this chromosomes
    chromosomes_file_path = 'chromosomes'
    
    def __init__(self,genNum,trueProb,reducedFeatureSubset,minFeatureNum=5,maxFeatureNum=12,gaType='hybrid') :
        self.chrom = np.full(genNum, False, dtype=bool)
        self.minFeatureNum = minFeatureNum
        self.maxFeatureNum = maxFeatureNum
        self.trueProb = trueProb # this has a conflict with Shakeri's method
        if gaType == 'hybrid' :
            self.feature_initialization(reducedFeatureSubset)
            if np.sum(self.chrom) < 5 :
                self.feature_initialization(np.delete(list(range(genNum)),reducedFeatureSubset))
        elif gaType == 'hybrid2' :
            r = np.random.randint(self.minFeatureNum,self.maxFeatureNum+1)
            #rankSum = np.sum(list(reducedFeatureSubset.values()))
            ranks =  reduced_feature_subset.copy()
            for _ in range(r):
                key = weighted_random_choice(ranks)
                self.chrom[int(key)] = True
        elif gaType == 'simple' :
            features = list(range(genNum))
            r = np.random.randint(self.minFeatureNum,self.maxFeatureNum+1)
            for i in range(r) :
                feature = np.random.choice(features)
                self.chrom[feature] = True
                features.remove(feature)
                
        self.fitness = None

    def weighted_random_choice(choices):
        maximum = sum(choices.values())
        pick = random.uniform(0, maximum)
        current = 0
        for key, value in choices.items():
            current += value
            if current > pick:
                choices.pop(key,None)
                return key
    def __iter__(self) :
        return np.nditer(self.chrom)
    
    def feature_initialization(self,chosenFeatures) :
        for i in chosenFeatures :
            if np.random.random() < self.trueProb :
                self.chrom[i] = True
                if np.sum(self.chrom) >= self.maxFeatureNum :
                    break   
        
    def mutate(self) :
        self.chrom = np.vectorize(lambda x:x if random() > (1/len(self.chrom)) else not x)(self.chrom)
        
    @staticmethod
    def keras_model(input_dim,hiddenNum=40,lr=.1,m=.5) :
        model = Sequential()
        model.add(Dense(hiddenNum, input_dim=input_dim, kernel_initializer='normal', activation='sigmoid'))
        # model.add(Dense(hiddenNum, kernel_initializer='normal', activation='selu'))
        model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
        sgd = SGD(lr=lr, momentum=m)
        ADAM = adam(lr=lr)
        # loss could be "mse" too
        model.compile(loss='binary_crossentropy',metrics=['accuracy','binary_accuracy'],optimizer=sgd)
        return model

    def calculateFitness(self,X,Y,trainCycles=450,lr=.1,m=.5) :
        """Train one layer feedforward neural network
        Args :
           X : training data
           Y : training label
           hiddenNum : number of hidden units of hidden layer
           trainCycles : number of training cycles
           lr : learning rate of nueral network
           m: momentum of neural network
        Returns :
           'float' accuracy
        """
        print('self.chrom',self.chrom)
        sel_features = np.array(self.chrom).astype(np.int32)
        print('sel_features',sel_features)
        hiddenNum = len(self.chrom) + np.sum(sel_features)
        string_arr = ''.join(map(str, 1*self.chrom))
        sum_val_acc = 0
        p_X = X[:,sel_features==1]
        if string_arr not in Chromosome.chromosomes :
            for train_index, test_index in skf.split(X, Y):
                model = self.keras_model(np.sum(sel_features),int(hiddenNum),lr,m)
                hist = model.fit(p_X[train_index,:],Y[train_index,:], validation_data=(p_X[test_index,:],Y[test_index,:]),epochs=int(trainCycles),batch_size=int(X.shape[0]),verbose=0)
                ev = model.evaluate(p_X[test_index,:],Y[test_index,:],verbose=0)
                sum_val_acc += ev[1]
                del model
            self.fitness = Chromosome.chromosomes[string_arr] = sum_val_acc/10
        else :
            self.fitness = Chromosome.chromosomes[string_arr]
        return Chromosome.chromosomes[string_arr]

In [36]:
class Population :
    
    def __init__(self,popSize,genNum,reducedFeatureSubset,minFeatureNum=5,maxFeatureNum=12,trueProb=.5,tPressure=.1,gaType='hybrid') :
        self.pop = np.array([])
        self.reducedFeatureSubset = reducedFeatureSubset
        self.minFeatureNum = minFeatureNum
        self.maxFeatureNum = maxFeatureNum
        self.popSize = popSize
        self.genNum = genNum
        self.gaType = gaType
        self.trueProb = trueProb
        self.tPressure = tPressure
        self.bestFitness = float('-inf')
        self.tSize = int(popSize*tPressure)
        if gaType != 'hybird2':
            for _ in range(popSize) : # initializing chromosome
                self.pop = np.append(self.pop,Chromosome(genNum,trueProb,reducedFeatureSubset,minFeatureNum,maxFeatureNum,gaType))
        else:
            for _ in range(popSize/2) : # initializing chromosome
                self.pop = np.append(self.pop,Chromosome(genNum,trueProb,reducedFeatureSubset,minFeatureNum,maxFeatureNum,gaType))
            for _ in range(popSize/2) : # initializing chromosome
                self.pop = np.append(self.pop,Chromosome(genNum,trueProb,reducedFeatureSubset,minFeatureNum,maxFeatureNum,'simple'))
    def __iter__(self) :
        return np.nditer(self.pop)
    
    def mutate(self) :
        for chrom in self.pop :
            chrom.mutate()
    
    def crossover(self,prob,cType='Uniform') :
        newPop = Population(int(len(self.pop)/4)*2,self.genNum,self.reducedFeatureSubset,self.minFeatureNum,self.maxFeatureNum,self.trueProb,self.tPressure)
        for i in range(int(len(self.pop)/4)) :
            rand_prob = np.random.random()
            first = self.tournament(self.tSize).chrom 
            second = self.tournament(self.tSize).chrom
            if cType == 'OnePoint' :     
                if rand_prob > prob :
                    point = random.randint(len(first))
                    newPop.pop[i*2].chrom = np.append(first[:point] + second[point:])
                    newPop.pop[i*2+1].chrom = np.append(first[:point] + second[point:])
            elif cType == 'Uniform' :
                for j in range(len(first)) :
                    if np.random.random() > .5 :
                        newPop.pop[i*2].chrom[j] = first[j]
                        newPop.pop[i*2+1].chrom[j] = second[j]
                    else :
                        newPop.pop[i*2].chrom[j] = second[j]
                        newPop.pop[i*2+1].chrom[j] = first[j] 
            else :
                newPop.pop[i*2].chrom = first[:]
                newPop.pop[i*2+1].chrom = second[:]
        return newPop
    
    def tournament(self,k) :
        return max(np.random.choice(self.pop,k,False),key=lambda c:c.fitness)
    
    def selection(self,newIndiv) :
        self.pop = sorted(self.pop,key=lambda c: c.fitness)
        for i in range(len(newIndiv)) :
            self.pop[i] = newIndiv[i]
    
    def calculateFitness(self,X,Y,trainCycles=450,lr=.1,m=.5) :
        for chrom in self.pop :
            fitness = chrom.calculateFitness(X,Y,int(trainCycles),lr,m)
            if fitness > self.bestFitness :         
                self.bestFitness = fitness
            print(fitness)
        return self.bestFitness
    
    @staticmethod
    def save_chromosomes():
        ToolBox.save_item(Chromosome.chromosomes_file_path,Chromosome.chromosomes)
    
    @staticmethod
    def load_chromosomes():
        Chromosome.chromosomes = ToolBox.load_item(Chromosome.chromosomes_file_path)

In [37]:
def geneticAlgorithm(X,Y,generationNum,popSize,genNum,trueProb,tPressure,cProb,mProb,trainCycles,lr,m,reducedFeatureSubset,minFeatureNum=5,maxFeatureNum=12,gaType='hybrid') :
    bestFitnesses = []
    population = Population(popSize,genNum,reducedFeatureSubset,minFeatureNum,maxFeatureNum,trueProb,tPressure,gaType)
    bestFitness = population.calculateFitness(X,Y,trainCycles,lr,m)
    bestFitnesses.append(bestFitness)
    for generation in range(generationNum) :
        newPop = population.crossover(cProb)
        newPop.mutate()
        fitness = newPop.calculateFitness(X,Y,trainCycles,lr,m)
        bestFitness = fitness if bestFitness < fitness else bestFitness
        print('Best Chromosome in {} Generation : {}'.format(generation,bestFitness))
        population.save_chromosomes()
        population.selection(newPop.pop)
        for p in newPop.pop :
            print('f ',p.fitness)
        bestFitnesses.append(bestFitness)
    return bestFitnesses

In [ ]:
x_train = pd.read_excel(german_data,header=None)
y_train = pd.read_excel(german_label,header=None)
genetic_config['genNum'] = x_train.shape[1]
# Population.load_chromosomes()
gaBestFitnesses = geneticAlgorithm(np.array(x_train),np.array(y_train),reducedFeatureSubset=reduced_feature_subset,gaType='simple',**genetic_config,**nn_config)
# geneticAlgorithm(np.array(x_train),np.array(y_train),reducedFeatureSubset=reduced_feature_subset,**genetic_config,**nn_config)

self.chrom [False False False  True False  True  True  True False  True  True False
 False False  True  True  True False  True  True  True False False False
 False False False False False False]
sel_features [0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0]
0.66
self.chrom [False False False False  True False False  True False False False False
  True False  True  True False  True False False False False  True False
 False False False  True  True False]
sel_features [0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0]
0.6989999999999998
self.chrom [False  True  True False  True  True False False False  True False False
 False  True False  True False False False False  True  True False False
 False False  True False  True  True]
sel_features [0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1]
0.7000000000000001
self.chrom [ True  True False False False False  True False False  True False  True
  True  True  True  True  True False  True False False False Fa

In [ ]:
ToolBox.save_item("GA_BestFitnesses",gaBestFitnesses)

In [24]:
plt.plot(list(range(51)), gaBestFitnesses)
plt.xlabel('Generation')
plt.ylabel('Accuracy')
plt.title("Genetic Algorithm Accuracy through its Generations")
plt.show()

NameError: name 'gaBestFitnesses' is not defined

In [25]:
plt.plot(list(range(51)), gaBestFitnesses)
plt.xlabel('Generation')
plt.ylabel('Accuracy')
plt.title("Genetic Algorithm Accuracy through its Generations")
plt.show()

NameError: name 'gaBestFitnesses' is not defined

In [ ]:
gaBestFitnesses = geneticAlgorithm(np.array(x_train),np.array(y_train),reducedFeatureSubset=reduced_feature_subset,gaType='hybrid2',**genetic_config,**nn_config)